In [1]:
#upload json file here
#initialize all the random function to 0
import random
import numpy as np
import tensorflow as tf
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import os
import json
from zipfile import ZipFile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models

In [5]:
#setup kaggle API to colab
abhi=json.load(open("kaggle.json"))


In [1]:
os.environ['KAGGLE_USERNAME']=abhi['username']
os.environ['KAGGLE_KEY']=abhi['key']

NameError: name 'abhi' is not defined

In [8]:
#extract the zip file
f1=ZipFile('archive (3).zip')
f1.extractall()
f1.close()


: 

In [ ]:
base_dir="plantvillage dataset/color"
#setting the base directory to color folder
ipath="/content/plantvillage dataset/color/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG"
img=mpimg.imread(ipath)
plt.imshow(img)
plt.axis("off")

In [ ]:
img.shape
#make all the images uniform
img_size=224
batch_size=32
data_gen=ImageDataGenerator(rescale=1./255,validation_split=0.2)

#validation split is a testing size
train_generator=data_gen.flow_from_directory(base_dir,target_size=(img_size,img_size),batch_size=batch_size,class_mode="categorical",subset="training")
#validation split is a testing size
validation_generator=data_gen.flow_from_directory(base_dir,target_size=(img_size,img_size),batch_size=batch_size,class_mode="categorical",subset="validation")
#making of neural network
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(img_size,img_size,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dense(train_generator.num_classes,activation="softmax"))

model.summary()
np.array([[1,2,3]]).flatten()
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          epochs=5,validation_data=validation_generator,
          validation_steps=validation_generator.samples//batch_size)
model.save("abhi.h5")
#above line is saving the model
#extract the classes from data with indexes
abhi_classes={v:k for k,v in train_generator.class_indices.items()}
#saving key as index and value as disease in above line
json.dump(abhi_classes,open("abhi_classes.json","w"))
def load_preprocess_image(ipath,target_size=(224,224)):
  img=Image.open(ipath)
  img=img.resize(target_size)
  img_array=np.array(img)
  img_array=np.expand_dims(img_array,axis=0)
  img_array=img_array.astype("float32")/255.
  return img_array
#above cell is resizing of image
def predict_disease(model,ipath,class_indices):
  preprocess_img=load_preprocess_image(ipath)
  prediction=model.predict(preprocess_img)
  predicted_class_index=np.argmax(prediction,axis=1)[0]
  predicted_class=class_indices[predicted_class_index]
  return predicted_class
#axis=1 is pulling the data from the column,0 is top mo
#Get image path
i1="/content/plantvillage dataset/color/Blueberry___healthy/0137389a-9b78-4a94-afdb-c2b50861f89c___RS_HL 2347.JPG"
i2="/content/plantvillage dataset/color/Grape___Black_rot/00090b0f-c140-4e77-8d20-d39f67b75fcc___FAM_B.Rot 0376.JPG"

predict_disease(model,i2,abhi_classes)
predict_disease(model,"/content/leaf.jpeg",abhi_classes)